# XOR Learning

- Logical operation is learned.

1. A shallow network will be used to test the network.
2. A 2-layer network will be tested.

In [1]:
import numpy as np
xy = np.loadtxt ('train-xor.txt', unpack=True)
xdata = xy[0:-1].transpose() # 0 에서 끝-1 까지
ydata = xy[-1:].transpose()
#ydata.reshape(ydata.shape[0],1)
print ('xdata', xdata, '\n', 'ydata', ydata)
print ('xdata[0].shape=', xdata[0].shape, xdata.shape[0], xdata.shape[1])
print ('ydata.shape=', ydata.shape)

xdata [[ 0.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  0.]] 
 ydata [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]]
xdata[0].shape= (2,) 4 2
ydata.shape= (4, 1)


## Specifying Unknown input batch size!

- The input data may have unknown number of data. 
- The codes below shows how to specify such inputs/output data.
- Notice that they are all defined as 2D tensors

In [2]:
import tensorflow as tf

X = tf.placeholder (tf.float32, [None,xdata.shape[1]])#+list(xdata[0].shape))
print ([None] + list(xdata[0].shape))
Y = tf.placeholder (tf.float32, [None]+list(ydata[0].shape))
print ([None]+list(ydata[0].shape))

[None, 2]
[None, 1]


In [3]:

W = tf.Variable (tf.random_uniform([xdata.shape[1],1], -1., 1.))
h = tf.matmul (X, W)
xor_out = tf.div (1., 1.+tf.exp(-h)) # sigmoid

yxlog = Y * tf.log(xor_out) + (1-Y)*tf.log(1.-xor_out)
cost = -tf.reduce_mean ( yxlog )

In [4]:
feed_dict={X:xdata, Y:ydata}
with tf.Session() as ss:
    ss.run(tf.global_variables_initializer())
    print ('X=', ss.run(X, feed_dict=feed_dict))
    print ('Y=', ss.run(Y, feed_dict))
    print ('W=', ss.run(W, feed_dict))
    print ('X*W=', ss.run(tf.matmul(X,W), feed_dict))
    #print ('h=', ss.run(h,feed_dict))
    print ('xor_out=', ss.run(xor_out, feed_dict))
    #print (np.divide(1., 1.+np.exp(-np.array([0., 0.24, 0.8, 0]))))
    print ('log(xor)=', ss.run(tf.log(xor_out), feed_dict))
    print ('y*log(xor)=', ss.run(Y*tf.log(xor_out),feed_dict))
    print ('tf.multipy(y,log(xor))=', ss.run(tf.multiply(Y,tf.log(xor_out)),feed_dict))
    print ('(1-y)*log(1-xor)=', ss.run((1.-Y)*tf.log(1.-xor_out),feed_dict))
    print ('yxlog=', ss.run(yxlog, feed_dict))
    print ('cost=', ss.run(cost,feed_dict))

X= [[ 0.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  0.]]
Y= [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]]
W= [[-0.89389777]
 [ 0.34588337]]
X*W= [[ 0.        ]
 [ 0.34588337]
 [-0.89389777]
 [ 0.        ]]
xor_out= [[ 0.5       ]
 [ 0.58561897]
 [ 0.29030612]
 [ 0.5       ]]
log(xor)= [[-0.69314718]
 [-0.53508592]
 [-1.23681927]
 [-0.69314718]]
y*log(xor)= [[-0.        ]
 [-0.53508592]
 [-1.23681927]
 [-0.        ]]
tf.multipy(y,log(xor))= [[-0.        ]
 [-0.53508592]
 [-1.23681927]
 [-0.        ]]
(1-y)*log(1-xor)= [[-0.69314718]
 [-0.        ]
 [-0.        ]
 [-0.69314718]]
yxlog= [[-0.69314718]
 [-0.53508592]
 [-1.23681927]
 [-0.69314718]]
cost= 0.78955


In [5]:
# minimize
opt = tf.train.AdamOptimizer(learning_rate=0.01)
train = opt.minimize(cost)

In [6]:
def evaluate(ss, xor_out, feed_dict):
    pred = ss.run (xor_out, feed_dict={X:xdata, Y:ydata})
    print ('pred=', pred.transpose())
    cpred = ss.run( tf.equal(tf.floor(xor_out+0.5), Y), feed_dict )
    print ('cpred=', cpred.transpose())
    accuracy = tf.reduce_mean (tf.cast(cpred, 'float'))
    print ('accuracy=', accuracy.eval(feed_dict))

In [7]:
with tf.Session() as ss:
    ss.run(tf.global_variables_initializer())
    
    # data fitting
    # 
    for i in range(10000):
        ss.run (train, feed_dict={X:xdata, Y:ydata})
        if i%1000==0:
            print ('------------------------- ',
                   i, 
                   ss.run(cost, feed_dict={X:xdata, Y:ydata}))
            evaluate (ss, xor_out, feed_dict)
    # model test
    #
    evaluate(ss,xor_out, feed_dict)

-------------------------  0 0.698733
pred= [[ 0.5         0.61371624  0.39835376  0.5       ]]
cpred= [[False  True False False]]
accuracy= 0.25
-------------------------  1000 0.357079
pred= [[ 0.5         0.98532909  0.97312725  0.5       ]]
cpred= [[False  True  True False]]
accuracy= 0.5
-------------------------  2000 0.349718
pred= [[ 0.5         0.99551255  0.99195158  0.5       ]]
cpred= [[False  True  True False]]
accuracy= 0.5
-------------------------  3000 0.347968
pred= [[ 0.5         0.99799675  0.99643242  0.5       ]]
cpred= [[False  True  True False]]
accuracy= 0.5
-------------------------  4000 0.347292
pred= [[ 0.5         0.99896526  0.99816328  0.5       ]]
cpred= [[False  True  True False]]
accuracy= 0.5
-------------------------  5000 0.346971
pred= [[ 0.5         0.99942625  0.99898344  0.5       ]]
cpred= [[False  True  True False]]
accuracy= 0.5
-------------------------  6000 0.346803
pred= [[ 0.5         0.99966955  0.99941504  0.5       ]]
cpred= [[False 

# Now, 2-layer network

The network has now twoo layers. This will increase the capacity of the network and result in a correctly fitted model.

In [8]:
nhidd = 12
W1 = tf.Variable (tf.random_uniform([xdata.shape[1],nhidd], -1., 1.))
b1 = tf.Variable (tf.zeros([nhidd]), name='b1')
W2 = tf.Variable (tf.random_uniform([nhidd,1],-1.,-1.))
b2 = tf.Variable (tf.zeros([1]), name='b2')

h1 = tf.sigmoid( tf.matmul (X, W1) + b1)
xor_out = tf.sigmoid (tf.matmul(h1, W2) + b2)

yxlog = Y * tf.log(xor_out) + (1-Y)*tf.log(1.-xor_out)
cost = -tf.reduce_mean ( yxlog )

# minimize
opti = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train2 = opti.minimize(cost)

with tf.Session() as ss:
    ss.run(tf.global_variables_initializer())
        
    # data fitting
    # 
    for i in range(10000):
        ss.run (train2, feed_dict={X:xdata, Y:ydata})
        if i%1000==0:
            print ('------- data fitting evaluation ------')
            print ('iteraiton: ', i, 
                  ' cost= ', ss.run(cost, feed_dict={X:xdata, Y:ydata}))
            evaluate (ss, xor_out, feed_dict)            
            
    # model test
    #
    evaluate (ss, xor_out, feed_dict)

------- data fitting evaluation ------
iteraiton:  0  cost=  0.829651
pred= [[ 0.26970422  0.30640954  0.22153802  0.26970422]]
cpred= [[ True False False  True]]
accuracy= 0.5
------- data fitting evaluation ------
iteraiton:  1000  cost=  0.680716
pred= [[ 0.48619905  0.53650069  0.46378252  0.48619905]]
cpred= [[ True  True False  True]]
accuracy= 0.75
------- data fitting evaluation ------
iteraiton:  2000  cost=  0.658058
pred= [[ 0.47312877  0.54800624  0.47276193  0.47312877]]
cpred= [[ True  True False  True]]
accuracy= 0.75
------- data fitting evaluation ------
iteraiton:  3000  cost=  0.620895
pred= [[ 0.44676167  0.56310868  0.48414886  0.44676167]]
cpred= [[ True  True False  True]]
accuracy= 0.75
------- data fitting evaluation ------
iteraiton:  4000  cost=  0.561479
pred= [[ 0.40717462  0.58887738  0.5113678   0.40717462]]
cpred= [[ True  True  True  True]]
accuracy= 1.0
------- data fitting evaluation ------
iteraiton:  5000  cost=  0.479513
pred= [[ 0.3522929   0.6266

# Deep Network Learning

In [14]:
nhidd = [5, 7, 9, 11, 9, 8, 4, 2, 1]
inputdim = xdata.shape[1]
inet = X
for hdim in nhidd:
    print ('inputdim = {}  hdim= {}'.format(inputdim, hdim))
    w = tf.Variable (tf.random_uniform([inputdim,hdim], -1., 1.))
    print ('w defined: [{}x{}]'.format(inputdim,hdim))
    b = tf.Variable (tf.zeros([hdim]))
    hh = tf.sigmoid (tf.matmul(inet, w) + b)
    
    inet = hh
    inputdim = hdim

xor_deep = inet
cost_deep = -tf.reduce_mean ( Y*tf.log(xor_deep) + (1.-Y)*tf.log(1.-xor_deep) )

# minimize
opti_deep = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_deep = opti_deep.minimize(cost_deep)

with tf.Session() as ss:
    ss.run(tf.global_variables_initializer())
        
    # data fitting
    # 
    for i in range(10000):
        ss.run (train_deep, feed_dict={X:xdata, Y:ydata})
        if i%1000==0:
            print ('------- data fitting evaluation ------')
            print ('iteraiton: ', i, 
                  ' cost= ', ss.run(cost_deep, feed_dict={X:xdata, Y:ydata}))
            evaluate (ss, xor_deep, feed_dict)            
            
    # model test
    #

    evaluate (ss, xor_deep, feed_dict)

inputdim = 2  hdim= 5
w defined: [2x5]
inputdim = 5  hdim= 7
w defined: [5x7]
inputdim = 7  hdim= 9
w defined: [7x9]
inputdim = 9  hdim= 11
w defined: [9x11]
inputdim = 11  hdim= 9
w defined: [11x9]
inputdim = 9  hdim= 8
w defined: [9x8]
inputdim = 8  hdim= 4
w defined: [8x4]
inputdim = 4  hdim= 2
w defined: [4x2]
inputdim = 2  hdim= 1
w defined: [2x1]
------- data fitting evaluation ------
iteraiton:  0  cost=  0.712688
pred= [[ 0.59788823  0.59788769  0.59788859  0.59788823]]
cpred= [[False  True  True False]]
accuracy= 0.5
------- data fitting evaluation ------
iteraiton:  1000  cost=  0.693152
pred= [[ 0.50158507  0.5015834   0.50158632  0.50158507]]
cpred= [[False  True  True False]]
accuracy= 0.5
------- data fitting evaluation ------
iteraiton:  2000  cost=  0.693147
pred= [[ 0.50002587  0.50002426  0.50002724  0.50002587]]
cpred= [[False  True  True False]]
accuracy= 0.5
------- data fitting evaluation ------
iteraiton:  3000  cost=  0.693147
pred= [[ 0.50000083  0.49999917  0.